# RNASeq data analysis using RNASeqTools

Requirements:

* Julia 1.8.5
* RNASeqTools 0.3.4
* sra-tools 3.0.3
* samtools 1.6
* bwa-mem2 2.2.1

In this tutorial, we will first download a few files (genome, annotation, sequencing reads) with sra-tools.
We will then align the reads to the genome using bwa-mem2. We will then use the various Packages from BioJulia
through the interface of RNASeqTools to load the alignments file, compute the coverage from it and do some
simple signal detection in the coverage.

In [1]:
using Pkg
Pkg.add(url="https://github.com/maltesie/RNASeqTools#v0.3.4")

using RNASeqTools

    Updating git-repo `https://github.com/maltesie/RNASeqTools#v0.3.4`
   Resolving package versions...
  No Changes to `~/BioTutorials/Project.toml`
  No Changes to `~/BioTutorials/Manifest.toml`


## Downloading the data

Our data will be from V. Cholerae, we need a genome and its annotation: First go to [NCBI](https://www.ncbi.nlm.nih.gov/data-hub/genome/GCF_013085075.1/),
after clicking Download, select .fasta and .gff and click Download again. Then extract the content of the archive
directory /ncbi_dataset/data/GCF_013085075.1/ into the folder containing this notebook. The folder should then
look like:


In [2]:
# RNASeqTools_TSS
#   ├── GCF_013085075.1_ASM1308507v1_genomic.fna
#   ├── genomic.gff
#   ├── Manifest.toml
#   ├── Project.toml
#   ├── rnaseqtools_tss.ipynb
#   └── rnaseqtools_tss.jl

NCBI stores reads in its internal format SRA, we can use prefetch and fastq-dump from the sra-tools to
download reads and convert them to the fastq format. The reads in sample SRR1602510 come from a TEX-treated
sample and can be used to identify primary transcription start sites.

In [3]:
download_prefetch(["SRR1602510"], @__DIR__)


2023-04-16T14:16:23 prefetch.3.0.2: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2023-04-16T14:16:23 prefetch.3.0.2: 1) Downloading 'SRR1602510'...
2023-04-16T14:16:23 prefetch.3.0.2: SRA Normalized Format file is being retrieved, if this is different from your preference, it may be due to current file availability.
2023-04-16T14:16:23 prefetch.3.0.2:  Downloading via HTTPS...
2023-04-16T14:16:58 prefetch.3.0.2:  HTTPS download succeed
2023-04-16T14:16:59 prefetch.3.0.2:  'SRR1602510' is valid
2023-04-16T14:16:59 prefetch.3.0.2: 1) 'SRR1602510' was downloaded successfully
Read 6686697 spots for ~/BioTutorials/RNASeqTools_TSS/SRR1602510.sra
Written 6686697 spots for ~/BioTutorials/RNASeqTools_TSS/SRR1602510.sra


## Aligning reads with bwa-mem

RNASeqTools provides a wrapper to bwa-mem2. We will use the genome file from the NCBI dataset downloaded earlier.
align_mem also calls samtools to sort and index the .bam file and creates a .log file containing a useful report
from samtools stats

In [4]:
align_mem(joinpath(@__DIR__, "SRR1602510_1.fastq.gz"), joinpath(@__DIR__, "GCF_013085075.1_ASM1308507v1_genomic.fna"))

[bwa_index] Pack FASTA... 0.02 sec
* Entering FMI_search
init ticks = 481045140
ref seq len = 8170862
binary seq ticks = 273930990
build suffix-array ticks = 3818121870
ref_seq_len = 8170862
count = 0, 2146471, 4085431, 6024391, 8170862
BWT[5860163] = 4
CP_SHIFT = 6, CP_MASK = 63
sizeof CP_OCC = 64
pos: 1021358, ref_seq_len__: 1021357
max_occ_ind = 127669
build fm-index ticks = 1029061770
Total time taken: 1.9117
-----------------------------
Executing in SSE4.2 mode!!
-----------------------------
* SA compression enabled with xfactor: 8
* Ref file: ~/BioTutorials/RNASeqTools_TSS/GCF_013085075.1_ASM1308507v1_genomic.fna
* Entering FMI_search
* Index file found. Loading index from ~/BioTutorials/RNASeqTools_TSS/GCF_013085075.1_ASM1308507v1_genomic.fna.bwt.2bit.64
* Reference seq len for bi-index = 8170863
* sentinel-index: 5860163
* Count:
0,	1
1,	2146472
2,	4085432
3,	6024392
4,	8170863

* Reading other elements of the index from files ~/BioTutorials/RNASeqTools_TSS/GCF_013085075.1_AS

Base.ProcessChain(Base.Process[Process(`samtools index ~/BioTutorials/RNASeqTools_TSS/SRR1602510_1.bam`, ProcessExited(0)), Process(`samtools stats ~/BioTutorials/RNASeqTools_TSS/SRR1602510_1.bam`, ProcessExited(0))], Base.DevNull(), Base.DevNull(), Base.DevNull())

## Annotation of alignments and exploration of the data

To annotate the alignments, we need to read the alignments file and the annotation file, then we can
explore the annotated alignments.

In [5]:
#read features with types ["rRNA", "tRNA", "CDS"] from the .gff file nad use ["ID", "Name", "locus_tag"] parameters to name the feature
ann = Features(joinpath(@__DIR__, "genomic.gff"), ["rRNA", "tRNA", "CDS"]; name_keys=["ID", "Name", "locus_tag"])

#add annotations for the regions upstream ("5UTR") and downstream ("3UTR") of each "CDS"
addutrs!(ann; cds_type="CDS", five_type="5UTR", five_length=200, three_type="3UTR", three_length=100)

#fill all regions without annotation with "IGR"
addigrs!(ann; igr_type="IGR")

alignments = AlignedReads(joinpath(@__DIR__, "SRR1602510_1.bam"))

#annotate each alignment with the feature with the largest overlap.
annotate!(alignments, ann)

println("The first 5 alignments with annotation:\n")
for (i, aln) in enumerate(alignments)
    show(aln)
    i>=5 && break
end

#count for each annotation type the number of alignments with a matching annotation
counter = Dict(t=>0 for t in types(ann))
for aln in alignments
    for fragment in aln
        counter[type(fragment)] += 1
    end
end
println("\nFrom $(length(alignments)) mapped fragments, $(counter["IGR"]) map to intergenic regions, $(counter["rRNA"]) to ribosomal RNA and $(counter["tRNA"]) to tRNAs.")

The first 5 alignments with annotation:

Single Alignment with 1 part(s):
   [1, 59] on read1 - [1929524, 1929582] on NZ_CP047295.1 (-) with edit distance 0 - IGR:cds-WP_001048829.1:cds-WP_001194704.1 (69% in annotation)
Single Alignment with 1 part(s):
   [1, 41] on read1 - [2589427, 2589467] on NZ_CP047295.1 (+) with edit distance 0 - IGR:cds-WP_001129565.1:cds-WP_001068472.1 (100% in annotation)
Single Alignment with 1 part(s):
   [1, 96] on read1 - [2807000, 2807095] on NZ_CP047295.1 (-) with edit distance 0 - rRNA:rna-GTF72_RS12795 (99% in annotation)
Single Alignment with 1 part(s):
   [1, 100] on read1 - [1031772, 1031871] on NZ_CP047296.1 (+) with edit distance 0 - IGR:cds-WP_001882860.1:cds-WP_001132600.1 (59% in annotation)
Single Alignment with 1 part(s):
   [1, 100] on read1 - [486446, 486545] on NZ_CP047295.1 (+) with edit distance 0 - 5UTR:cds-WP_000567318.1 (100% in annotation)

From 5899560 mapped fragments, 2002875 map to intergenic regions, 2074559 to ribosomal RNA an

## Computation of coverage and potential TSS from it

In [6]:
#coverage is computed with a default normalization of 1/Million reads
forward_file, reverse_file = compute_coverage(joinpath(@__DIR__, "SRR1602510_1.bam"))
coverage = Coverage(forward_file, reverse_file)

#if we dont want to put the coverage into files first, we can compute it directly from the alignments:
coverage = Coverage(alignments)

RNASeqTools.Coverage on 2 reference sequences

Now we can call peaks in the difference along the coverage as positions of putative TSS. The maxdiffpositions
function computes maxima in the difference along the coverage with a minimum step parameter and a
minimum ratio towards the background within a certain region around the maximum.

In [7]:
tss_features = maxdiffpositions(coverage; min_diff=5, min_ratio=1.3, compute_within=3)
println("Found $(length(tss_features)) putative TSS\nThe first 5 TSS according to the set parameters:\n")
for (i, tss) in enumerate(tss_features)
    show(tss)
    i>=5 && break
end

#maxdiffpositions returns a Features struct that can be saved to disc into a .gff file:
write(joinpath(@__DIR__, "tss.gff"), tss_features)

Found 5138 putative TSS
The first 5 TSS according to the set parameters:

Feature: [159, 159] on NZ_CP047295.1 (-) - annotation: DIFF:reverse_1
parameters: Name=reverse_1;diff=10.509258317569445;from=NA;height=10.509258317569445
Feature: [1424, 1424] on NZ_CP047295.1 (+) - annotation: DIFF:forward_1
parameters: Name=forward_1;diff=9.153224986270164;from=NA;height=11.187274983219089
Feature: [2440, 2440] on NZ_CP047295.1 (+) - annotation: DIFF:forward_2
parameters: Name=forward_2;diff=43.56257076798948;from=NA;height=45.08810826570117
Feature: [2891, 2891] on NZ_CP047295.1 (+) - annotation: DIFF:forward_3
parameters: Name=forward_3;diff=83.90456237414317;from=NA;height=87.8031582016286
Feature: [3242, 3242] on NZ_CP047295.1 (-) - annotation: DIFF:reverse_2
parameters: Name=reverse_2;diff=12.54330831451837;from=NA;height=12.712812480930781


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*